<a href="https://colab.research.google.com/github/Naveed2204016/PoliMemeDecode/blob/main/LastOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c poli-meme-decode-cuet-cse-fest

 76% 275M/364M [00:00<00:00, 989MB/s] 
100% 364M/364M [00:03<00:00, 96.3MB/s]


In [ ]:
!kaggle competitions download -c poli-meme-decode-cuet-cse-fest

poli-meme-decode-cuet-cse-fest.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/poli-meme-decode-cuet-cse-fest.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
# =========================
# 0. Installs (run once)
# =========================

!pip install -q --upgrade pip

!pip install -q "transformers>=4.40.0" \
                 "sentence-transformers>=2.2.2" \
                 "easyocr" \
                 "tqdm" \
                 "datasets" \
                 "huggingface_hub"



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.9 MB/s eta 0:00:00


In [ ]:
# =========================
# 1. Imports & Setup
# =========================
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModel
import easyocr
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
# =========================
# 2. Paths and CSVs
# =========================
TRAIN_CSV = "/content/PoliMemeDecode/Train/Train.csv"
TRAIN_IMG_DIR = "/content/PoliMemeDecode/Train/Image/"
TEST_CSV = "/content/PoliMemeDecode/Test/Test.csv"
TEST_IMG_DIR = "/content/PoliMemeDecode/Test/Image/"

train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

# Add full file paths
train_df["filepath"] = train_df["Image_name"].apply(lambda x: os.path.join(TRAIN_IMG_DIR, x))
test_df["filepath"]  = test_df["Image_name"].apply(lambda x: os.path.join(TEST_IMG_DIR, x))

label_map = {"Political": 1, "NonPolitical": 0}
train_df["label"] = train_df["Label"].map(label_map)

print("Train examples:", len(train_df))
print("Test examples:", len(test_df))


Train examples: 2860
Test examples: 330


In [ ]:
# =========================
# 3. Load OCR + Pretrained Models (PyTorch)
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# EasyOCR reader (Bangla + English)
reader = easyocr.Reader(['bn','en'], gpu=(device=="cuda"))

# CLIP (image encoder)
IMAGE_MODEL_NAME = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(IMAGE_MODEL_NAME)
clip_model = CLIPModel.from_pretrained(IMAGE_MODEL_NAME).to(device)
clip_model.eval()

# XLM-R for text encoding
TEXT_MODEL = "xlm-roberta-base"
tokenizer_text = AutoTokenizer.from_pretrained(TEXT_MODEL)
text_model = AutoModel.from_pretrained(TEXT_MODEL).to(device)
text_model.eval()


device: cuda
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine

In [ ]:
# =========================
# 4. Helper functions (safe, batched)
# =========================
import math
from PIL import Image

torch.set_grad_enabled(False)

def extract_ocr_text(path):
    """Robust OCR wrapper returning a short cleaned string."""
    try:
        texts = reader.readtext(path, detail=0)
        joined = " ".join(texts)
        if len(joined.strip()) == 0:
            return "no_text"
        # minimal cleaning
        joined = joined.replace("\n", " ").strip()
        return joined
    except Exception:
        return "no_text"

def compute_clip_image_embeddings(paths, batch_size=16):
    """Compute CLIP image embeddings (N, dim) in batches. Returns numpy array."""
    all_embs = []
    for i in tqdm(range(0, len(paths), batch_size), desc="Image emb batches"):
        batch_paths = paths[i : i + batch_size]
        pil_imgs = [Image.open(p).convert("RGB") for p in batch_paths]
        inputs = clip_processor(images=pil_imgs, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = clip_model.get_image_features(**inputs)  # (B, dim)
            emb = outputs.cpu().numpy()
        all_embs.append(emb)
    return np.vstack(all_embs)

def compute_text_embeddings(texts, batch_size=16, max_length=128):
    """Compute CLS embeddings for texts (XLM-R output) in batches. Returns numpy array."""
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Text emb batches"):
        batch_texts = texts[i : i + batch_size]
        enc = tokenizer_text(batch_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
        input_ids = enc["input_ids"].to(device)
        attention_mask = enc["attention_mask"].to(device)
        with torch.no_grad():
            outputs = text_model(input_ids=input_ids, attention_mask=attention_mask)
            cls = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embs.append(cls)
    return np.vstack(all_embs)


In [ ]:
# =========================
# 5. OCR for TRAIN (batched)
# =========================
train_texts = []
for p in tqdm(train_df["filepath"].values, desc="OCR Train"):
    train_texts.append(extract_ocr_text(p))
train_df["ocr_text"] = train_texts
# quick check
train_df[["Image_name","ocr_text"]].head()


OCR Train: 100%|██████████| 2860/2860 [22:09<00:00,  2.15it/s]


,Image_name,ocr_text
0,train0001.jpg,হরেক রকম চোর দিয়া বানাইছো মেলা পিসি তোমার দরব...
1,train0002.jpg,বর যাত্রী যাওয়ার টাইমে যখব গাড়িতে জায়গা কম পড়...
2,train0003.jpg,তুমিকেখা নোবলল বুকে অনেক ব্যখাহেয় TARLBAU; ঠিক...
3,train0004.jpg,যখন ভল্লুক এসে পড়ে এবং আপন এবং আপনার বেস্টফ়র...
4,train0005.jpg,বন্ধুযখন তার গফকেকাজিন বলে পরিচয় দেয় [41#4 {...


In [ ]:
 # =========================
# 6. Compute text embeddings for TRAIN (batch safe)
# =========================
train_text_embeddings = compute_text_embeddings(train_df["ocr_text"].tolist(), batch_size=16, max_length=128)
print("train text emb shape:", train_text_embeddings.shape)


Text emb batches: 100%|██████████| 179/179 [00:15<00:00, 11.49it/s]

train text emb shape: (2860, 768)


In [ ]:
# =========================
# 7. Compute image embeddings for TRAIN (batch safe)
# =========================
train_image_embeddings = compute_clip_image_embeddings(train_df["filepath"].tolist(), batch_size=16)
print("train image emb shape:", train_image_embeddings.shape)


Image emb batches: 100%|██████████| 179/179 [01:18<00:00,  2.28it/s]

train image emb shape: (2860, 512)


In [ ]:
# =========================
# 8. Train/Validation split (indices so alignments keep)
# =========================
indices = np.arange(len(train_df))
train_idx, val_idx = train_test_split(indices, test_size=0.15, random_state=42, stratify=train_df["label"].values)

X_img_train = train_image_embeddings[train_idx]
X_txt_train = train_text_embeddings[train_idx]
y_train     = train_df["label"].values[train_idx]

X_img_val = train_image_embeddings[val_idx]
X_txt_val = train_text_embeddings[val_idx]
y_val     = train_df["label"].values[val_idx]

print("Shapes train:", X_img_train.shape, X_txt_train.shape, y_train.shape)
print("Shapes val:  ", X_img_val.shape, X_txt_val.shape, y_val.shape)


Shapes train: (2431, 512) (2431, 768) (2431,)
Shapes val:   (429, 512) (429, 768) (429,)


In [ ]:
# =========================
# 9. Build Keras Fusion Classifier
# =========================
img_dim = X_img_train.shape[1]
txt_dim = X_txt_train.shape[1]

img_in = Input(shape=(img_dim,), name="image_input")
txt_in = Input(shape=(txt_dim,), name="text_input")

p_img = Dense(512, activation="relu")(img_in)
p_img = BatchNormalization()(p_img)
p_img = Dropout(0.3)(p_img)

p_txt = Dense(512, activation="relu")(txt_in)
p_txt = BatchNormalization()(p_txt)
p_txt = Dropout(0.3)(p_txt)

merged = Concatenate()([p_img, p_txt])
x = Dense(512, activation="relu")(merged)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
out = Dense(1, activation="sigmoid")(x)

fusion_model = Model(inputs=[img_in, txt_in], outputs=out)
fusion_model.compile(optimizer=keras.optimizers.Adam(5e-6),
                     loss="binary_crossentropy", metrics=["accuracy"])
fusion_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_input          │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │    262,656 │ image_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │    393,728 │ text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512)       │      2,048 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512)       │      2,048 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1024)      │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 512)       │    524,800 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512)       │      2,048 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     65,664 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │        129 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,253,121 (4.78 MB)

 Trainable params: 1,250,049 (4.77 MB)

 Non-trainable params: 3,072 (12.00 KB)

In [ ]:
# =========================
# 10. Compute class weights (optional) and fit
# =========================
from sklearn.utils import class_weight
cw = class_weight.compute_class_weight("balanced", classes=np.unique(train_df["label"].values), y=train_df["label"].values)
class_weights = {i: float(cw[i]) for i in range(len(cw))}
print("class_weights:", class_weights)

history = fusion_model.fit(
    [X_img_train, X_txt_train], y_train,
    validation_data=([X_img_val, X_txt_val], y_val),
    epochs=8,
    batch_size=32,
    class_weight=class_weights
)


class_weights: {0: 0.7125062282012955, 1: 1.6764361078546308}
Epoch 1/8
76/76 ━━━━━━━━━━━━━━━━━━━━ 13s 58ms/step - accuracy: 0.4778 - loss: 0.9574 - val_accuracy: 0.4172 - val_loss: 0.7173
Epoch 2/8
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5032 - loss: 0.9233 - val_accuracy: 0.6597 - val_loss: 0.6489
Epoch 3/8
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5128 - loss: 0.8693 - val_accuracy: 0.7156 - val_loss: 0.6165
Epoch 4/8
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5152 - loss: 0.8665 - val_accuracy: 0.7110 - val_loss: 0.6000
Epoch 5/8
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5640 - loss: 0.7712 - val_accuracy: 0.6876 - val_loss: 0.5928
Epoch 6/8
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5682 - loss: 0.7792 - val_accuracy: 0.6900 - val_loss: 0.5893
Epoch 7/8
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5983 - loss: 0.7254 - val_accuracy: 0.6923 - val_loss: 0.5811
Epoch 8/8
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 

In [ ]:
# =========================
# 11. OCR + embeddings for TEST (batched)
# =========================
test_texts = []
for p in tqdm(test_df["filepath"].values, desc="OCR Test"):
    test_texts.append(extract_ocr_text(p))
test_df["ocr_text"] = test_texts

test_text_embeddings = compute_text_embeddings(test_df["ocr_text"].tolist(), batch_size=16, max_length=128)
print("test text emb shape:", test_text_embeddings.shape)

test_image_embeddings = compute_clip_image_embeddings(test_df["filepath"].tolist(), batch_size=16)
print("test image emb shape:", test_image_embeddings.shape)


Text emb batches: 100%|██████████| 21/21 [00:01<00:00, 12.74it/s]


test text emb shape: (330, 768)


Image emb batches: 100%|██████████| 21/21 [00:13<00:00,  1.60it/s]

test image emb shape: (330, 512)


In [ ]:
# =========================
# 12. Predict & Create submission.csv
# =========================
pred_probs = fusion_model.predict([test_image_embeddings, test_text_embeddings], batch_size=64)
preds = (pred_probs.flatten() >= 0.5).astype(int)

inv_label_map = {1: "Political", 0: "NonPolitical"}
pred_labels = [inv_label_map[int(x)] for x in preds]

submission = pd.DataFrame({
    "Image_name": test_df["Image_name"],
    "Label": pred_labels
})
submission.to_csv("submission5.csv", index=False)
print("submission.csv")
submission.head()


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
submission.csv


,Image_name,Label
0,test0001.jpg,Political
1,test0002.jpg,NonPolitical
2,test0003.jpg,Political
3,test0004.jpg,Political
4,test0005.jpg,Political
